In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


C:\Users\Crystal\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [2]:
df = pd.read_csv('useducation-Cleaned2.csv')

### 1) Create a new score variable from the weighted averages of all score variables in the datasets. Notice that the number of students in the 4th grade isn't the same as the number of students in the 8th grade. So, you should appropriately weigh the scores!.

In [3]:
grade_cols = ['AVG_MATH_4_SCORE', 'AVG_MATH_8_SCORE', 'AVG_READING_4_SCORE', 'AVG_READING_8_SCORE']

In [4]:
df.columns

Index(['PRIMARY_KEY', 'STATE', 'YEAR', 'TOTAL_REVENUE', 'FEDERAL_REVENUE',
       'STATE_REVENUE', 'LOCAL_REVENUE', 'TOTAL_EXPENDITURE',
       'INSTRUCTION_EXPENDITURE', 'SUPPORT_SERVICES_EXPENDITURE',
       'OTHER_EXPENDITURE', 'CAPITAL_OUTLAY_EXPENDITURE', 'GRADES_PK_G',
       'GRADES_KG_G', 'GRADES_4_G', 'GRADES_8_G', 'GRADES_12_G',
       'GRADES_1_8_G', 'GRADES_9_12_G', 'GRADES_ALL_G', 'AVG_MATH_4_SCORE',
       'AVG_MATH_8_SCORE', 'AVG_READING_4_SCORE', 'AVG_READING_8_SCORE'],
      dtype='object')

In [5]:
df.head()[grade_cols]

,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,208.327876,252.187522,207.963517,260.276441
1,223.833455,277.643071,211.547154,258.859712
2,215.253932,265.366278,206.212716,262.169895
3,210.206028,256.312090,208.634458,264.619665
4,208.398961,260.892247,196.764414,269.197451


In [6]:
def get_wt_avg(row):
    four = row['GRADES_4_G'] * (row['AVG_MATH_4_SCORE'] + row['AVG_READING_4_SCORE'])/2
    eight = row['GRADES_8_G'] * (row['AVG_MATH_8_SCORE'] + row['AVG_READING_8_SCORE'])/2
    wt_avg = (four + eight)/(row['GRADES_4_G'] + row['GRADES_8_G'])
    return wt_avg

In [7]:
df['wt_avg'] = df.apply(lambda row: get_wt_avg(row) , axis = 1)

### 2) What are the correlations between this newly created score variable and the expenditure types? Which 1 of the expenditure types is more correlated than the others?

The expenditure with the highest correlation to the weighted scores is Support Services Expenditure, although instruction expenditure similarly correlated. 

In [67]:
df[['TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE', 'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE', 'CAPITAL_OUTLAY_EXPENDITURE', 'wt_avg']].corr()

,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,wt_avg
TOTAL_EXPENDITURE,1.000000,0.991256,0.993101,0.939809,0.930754,0.119924
INSTRUCTION_EXPENDITURE,0.991256,1.000000,0.976215,0.902637,0.894034,0.126793
SUPPORT_SERVICES_EXPENDITURE,0.993101,0.976215,1.000000,0.947094,0.915849,0.127076
OTHER_EXPENDITURE,0.939809,0.902637,0.947094,1.000000,0.916466,0.076439
CAPITAL_OUTLAY_EXPENDITURE,0.930754,0.894034,0.915849,0.916466,1.000000,0.051503
wt_avg,0.119924,0.126793,0.127076,0.076439,0.051503,1.000000


### 3) Now, apply PCA to the 4 expenditure types. How much of the total variance is explained by the 1st component?

In [43]:
finance_df = df[['TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE', 'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE', 'wt_avg']].dropna(axis = 0)

In [56]:
X = StandardScaler().fit_transform(finance_df.drop('wt_avg', axis = 1))
sklearn_pca = PCA(n_components=1)
Y_sklearn = sklearn_pca.fit_transform(X)

In [57]:
print('The total variance explained by the 1st component is {}'.format(sklearn_pca.explained_variance_ratio_[0]*100))

The total variance explained by the 1st component is 96.89521365307421


### 4) What is the correlation between the overall score variable and the 1st principal component?

In [59]:
finance_df['pca1'] = Y_sklearn

In [68]:
finance_df.corr()

,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,wt_avg,pca1
TOTAL_EXPENDITURE,1.000000,0.991256,0.993101,0.939809,0.119924,0.996865
INSTRUCTION_EXPENDITURE,0.991256,1.000000,0.976215,0.902637,0.126793,0.983274
SUPPORT_SERVICES_EXPENDITURE,0.993101,0.976215,1.000000,0.947094,0.127076,0.994854
OTHER_EXPENDITURE,0.939809,0.902637,0.947094,1.000000,0.076439,0.962033
wt_avg,0.119924,0.126793,0.127076,0.076439,1.000000,0.114603
pca1,0.996865,0.983274,0.994854,0.962033,0.114603,1.000000


The correlation is 0.114603.

### 5)  If you were to choose the best variables for your model, would you prefer using the 1st principal component instead of the expenditure variables? Why?

Correlations between each of the expenditure variables are highly correlated with each other (> 0.8).  This can create problems if using PCA.  In addition, the correlation between support services expenditure and 1st principal component are 0.127076 and 0.114603 respectively.  The correlation for the expenditure variable is higher.  These two combined points to the expenditure variables being the better choice.